I set all of the `enc_shape = es` for every instance of the autoencoder. So just set `es=2, es=4`... to get all of the autoencoders.

https://github.com/techshot25/Autoencoders/blob/master/simple-autoencoder.ipynb
This simple code shows you how to make an autoencoder using Pytorch. The idea is to bring down the number of dimensions (or reduce the feature space) using neural networks.

The idea is simple, let the neural network learn how to make the encoder and the decoder using the feature space as both the input and the output of the network.

### Import relevant packages

In [87]:
import torch
from torch import nn, optim
import numpy as np
from matplotlib import pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

In [88]:
import cython 

import skfda
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from skfda.exploratory.depth import ModifiedBandDepth, IntegratedDepth
from skfda.exploratory.visualization import Boxplot

from skfda.exploratory.visualization.clustering import (
    ClusterMembershipLinesPlot,
    ClusterMembershipPlot,
    ClusterPlot,
)
from skfda.ml.clustering import FuzzyCMeans, KMeans

from skfda.exploratory.visualization import FPCAPlot
from skfda.preprocessing.dim_reduction.feature_extraction import FPCA
from skfda.representation.basis import BSpline, Fourier, Monomial

### Read in data

In [89]:
X = pd.read_csv('Data/ML_GRF_stance_N.csv').values

In [90]:
Y = pd.read_csv('Data/AP_GRF_stance_N.csv').values

In [91]:
Z = pd.read_csv('Data/V_GRF_stance_N.csv').values

### Define functions and build Autoencoder

In [92]:
class Autoencoder(nn.Module):
    """Makes the main denoising auto

    Parameters
    ----------
    in_shape [int] : input shape
    enc_shape [int] : desired encoded shape
    """

    def __init__(self, in_shape, enc_shape):
        super(Autoencoder, self).__init__()
        
        self.encode = nn.Sequential(
            nn.Linear(in_shape, 128),
            nn.ReLU(True),
            nn.Dropout(0.2), #regularization
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(64, enc_shape),
        )
        
        self.decode = nn.Sequential(
            nn.BatchNorm1d(enc_shape),
            nn.Linear(enc_shape, 64),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(128, in_shape)
        )
        
    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)
        return x

In [93]:
def train(model, error, optimizer, n_epochs, x):
    model.train()
    for epoch in range(1, n_epochs + 1):
        optimizer.zero_grad()
        output = model(x)
        loss = error(output, x)
        loss.backward()
        optimizer.step()
        
        if epoch % int(0.1*n_epochs) == 0:
            print(f'epoch {epoch} \t Loss: {loss.item():.4g}')

In [94]:
#Defining a list of grid points
grid_points_100 = list(range(1,101))
grid_points_2990 = list(range(1,2991))

In [147]:
es = 6
ne = 1000

#### 1. GRF_ML

In [148]:
device_x = ('cuda' if torch.cuda.is_available() else 'cpu')

In [149]:
x = torch.from_numpy(X).to(device_x)

In [150]:
encoder_x = Autoencoder(in_shape=100, enc_shape=es).double().to(device_x)

error_x = nn.MSELoss()

optimizer_x = optim.Adam(encoder_x.parameters())

In [151]:
train(encoder_x, error_x, optimizer_x, ne, x)

epoch 100 	 Loss: 137.3
epoch 200 	 Loss: 108.9
epoch 300 	 Loss: 96.8
epoch 400 	 Loss: 93.63
epoch 500 	 Loss: 92.2
epoch 600 	 Loss: 90.77
epoch 700 	 Loss: 89.76
epoch 800 	 Loss: 88.57
epoch 900 	 Loss: 82.41
epoch 1000 	 Loss: 75.57


In [152]:
with torch.no_grad():
    encoded_x = encoder_x.encode(x)
    decoded_x = encoder_x.decode(encoded_x)
    mse_x = error_x(decoded_x, x).item()
    enc_x = encoded_x.cpu().detach().numpy()
    dec_x = decoded_x.cpu().detach().numpy()

#### 2. GRF_AP

In [153]:
device_y = ('cuda' if torch.cuda.is_available() else 'cpu')

In [154]:
y = torch.from_numpy(Y).to(device_y)

In [155]:
encoder_y = Autoencoder(in_shape=100, enc_shape=es).double().to(device_y)

error_y = nn.MSELoss()

optimizer_y = optim.Adam(encoder_y.parameters())

In [156]:
train(encoder_y, error_y, optimizer_y, ne, y)

epoch 100 	 Loss: 1076
epoch 200 	 Loss: 375.9
epoch 300 	 Loss: 363.4
epoch 400 	 Loss: 356
epoch 500 	 Loss: 352.7
epoch 600 	 Loss: 350.1
epoch 700 	 Loss: 345.7
epoch 800 	 Loss: 343.3
epoch 900 	 Loss: 341.3
epoch 1000 	 Loss: 339.6


In [157]:
with torch.no_grad():
    encoded_y = encoder_y.encode(y)
    decoded_y = encoder_y.decode(encoded_y)
    mse_y = error_y(decoded_y, y).item()
    enc_y = encoded_y.cpu().detach().numpy()
    dec_y = decoded_y.cpu().detach().numpy()

#### 3. GRF_V

In [158]:
device_z = ('cuda' if torch.cuda.is_available() else 'cpu')

In [159]:
z = torch.from_numpy(Z).to(device_z)

In [160]:
encoder_z = Autoencoder(in_shape=100, enc_shape=es).double().to(device_z)

error_z = nn.MSELoss()

optimizer_z = optim.Adam(encoder_z.parameters())

In [161]:
train(encoder_z, error_z, optimizer_z, ne, z)

epoch 100 	 Loss: 3.288e+05
epoch 200 	 Loss: 1.521e+04
epoch 300 	 Loss: 1.26e+04
epoch 400 	 Loss: 1.215e+04
epoch 500 	 Loss: 1.16e+04
epoch 600 	 Loss: 1.141e+04
epoch 700 	 Loss: 1.143e+04
epoch 800 	 Loss: 1.133e+04
epoch 900 	 Loss: 1.114e+04
epoch 1000 	 Loss: 1.12e+04


In [162]:
with torch.no_grad():
    encoded_z = encoder_z.encode(z)
    decoded_z = encoder_z.decode(encoded_z)
    mse_z = error_z(decoded_z, z).item()
    enc_z = encoded_z.cpu().detach().numpy()
    dec_z = decoded_z.cpu().detach().numpy()

### Calculate the MSE

In [163]:
# Define function for MSE

def mse(array1,array2):
    diff = array1 - array2
    sqdiff = diff**2
    n = len(sqdiff)
    
    sum_sqdiff = sum(sqdiff)
    return sum_sqdiff/n

In [164]:
# GRF_ML

ML_GRF_reconst_array = dec_x.flatten()
ML_GRF_array = X.flatten()

In [165]:
mse(ML_GRF_array, ML_GRF_reconst_array)

75.35283382368439

In [166]:
# GRF_AP

AP_GRF_reconst_array = dec_y.flatten()
AP_GRF_array = Y.flatten()

In [167]:
mse(AP_GRF_array, AP_GRF_reconst_array)

338.5241130553682

In [168]:
# GRF_V

V_GRF_reconst_array = dec_z.flatten()
V_GRF_array = Z.flatten()

In [169]:
mse(V_GRF_array, V_GRF_reconst_array)

11269.839468769007